# 系統

In [1]:
import tkinter as tk
from tkinter import messagebox, simpledialog

# 病患資料結構
patients = []

# 遮罩姓名函數
def mask_name(name):
    if len(name) == 2:
        return name[0] + 'O'
    elif len(name) > 2:
        return name[0] + 'O'*(len(name)-2) + name[-1]
    return name

# 更新候診名單顯示
def update_display():
    patient_listbox.delete(0, tk.END)
    for idx, patient in enumerate(patients):
        display_text = f"{idx+1}. {mask_name(patient['name'])} | {patient['phone']} | {patient['status']}"
        patient_listbox.insert(tk.END, display_text)

# 新增掛號
def add_patient():
    name = simpledialog.askstring("新增掛號", "請輸入病患姓名:")
    phone = simpledialog.askstring("新增掛號", "請輸入聯絡電話:")
    if name and phone:
        patients.append({'name': name, 'phone': phone, 'status': '候診中'})
        update_display()

# 修改掛號狀態
def modify_status():
    selected = patient_listbox.curselection()
    if selected:
        idx = selected[0]
        status = simpledialog.askstring("修改狀態", "請輸入狀態(已到診/未到診/取消):")
        if status in ['已到診', '未到診', '取消']:
            patients[idx]['status'] = status
            update_display()
        else:
            messagebox.showwarning("錯誤", "狀態輸入不正確！")
    else:
        messagebox.showwarning("提示", "請先選擇病患！")

# 刪除掛號
def delete_patient():
    selected = patient_listbox.curselection()
    if selected:
        idx = selected[0]
        del patients[idx]
        update_display()
    else:
        messagebox.showwarning("提示", "請先選擇病患！")

# 主視窗
def main():
    global patient_listbox
    root = tk.Tk()
    root.title("診所掛號系統")

    # 候診名單顯示
    tk.Label(root, text="即時候診名單", font=('Arial', 16)).pack()
    patient_listbox = tk.Listbox(root, width=50, height=15, font=('Arial', 14))
    patient_listbox.pack()

    # 功能按鈕
    btn_frame = tk.Frame(root)
    btn_frame.pack(pady=10)

    tk.Button(btn_frame, text="新增掛號", command=add_patient, width=12).grid(row=0, column=0, padx=5)
    tk.Button(btn_frame, text="修改狀態", command=modify_status, width=12).grid(row=0, column=1, padx=5)
    tk.Button(btn_frame, text="刪除掛號", command=delete_patient, width=12).grid(row=0, column=2, padx=5)

    update_display()
    root.mainloop()

if __name__ == "__main__":
    main()


In [2]:
import tkinter as tk
from tkinter import ttk, messagebox
from datetime import datetime

patients = []

def mask_name(name: str) -> str:
    if len(name) == 2:
        return name[0] + 'O'
    elif len(name) > 2:
        return name[0] + 'O' * (len(name) - 2) + name[-1]
    return name

def update_display(filter_status: str = "全部") -> None:
    patient_listbox.delete(*patient_listbox.get_children())
    count = 0
    for idx, p in enumerate(patients):
        if filter_status == "全部" or p['status'] == filter_status:
            patient_listbox.insert('', 'end', values=(
                idx + 1, mask_name(p['name']), p['phone'], p['status'], p['time']))
            count += 1
    count_label.config(text=f"目前候診總人數：{count}")

def open_add_window() -> None:
    win = tk.Toplevel(root)
    win.title("新增掛號")
    win.geometry("300x200")

    tk.Label(win, text="姓名").pack(pady=5)
    name_entry = tk.Entry(win)
    name_entry.pack()

    tk.Label(win, text="聯絡電話").pack(pady=5)
    phone_entry = tk.Entry(win)
    phone_entry.pack()

    def submit():
        name = name_entry.get()
        phone = phone_entry.get()
        if name and phone:
            patients.append({
                'name': name,
                'phone': phone,
                'status': '候診中',
                'time': datetime.now().strftime('%H:%M:%S')
            })
            update_display(status_var.get())
            win.destroy()
        else:
            messagebox.showwarning("錯誤", "請填寫所有欄位")

    tk.Button(win, text="確認新增", command=submit).pack(pady=10)

def modify_status() -> None:
    selected = patient_listbox.selection()
    if selected:
        idx = int(patient_listbox.item(selected[0])['values'][0]) - 1
        current_status = patients[idx]['status']

        def set_status(new_status: str):
            patients[idx]['status'] = new_status
            update_display(status_var.get())
            status_win.destroy()

        status_win = tk.Toplevel(root)
        status_win.title("修改狀態")
        tk.Label(status_win, text=f"目前狀態：{current_status}", font=("Arial", 12)).pack(pady=10)
        for s in ['已到診', '未到診', '取消']:
            tk.Button(status_win, text=s, command=lambda s=s: set_status(s), width=10).pack(pady=5)
    else:
        messagebox.showinfo("提示", "請選擇一位病患")

def delete_patient() -> None:
    selected = patient_listbox.selection()
    if selected:
        idx = int(patient_listbox.item(selected[0])['values'][0]) - 1
        del patients[idx]
        update_display(status_var.get())
    else:
        messagebox.showwarning("提示", "請先選擇病患！")

def change_filter(*args):
    update_display(status_var.get())

root = tk.Tk()
root.title("診所掛號系統")
root.geometry("800x500")

# 狀態篩選
status_var = tk.StringVar(value="全部")
status_menu = ttk.OptionMenu(root, status_var, "全部", "全部", "候診中", "已到診", "未到診", "取消", command=change_filter)
status_menu.pack(pady=10)

# 候診清單顯示（Treeview）
columns = ('序號', '姓名', '電話', '狀態', '掛號時間')
patient_listbox = ttk.Treeview(root, columns=columns, show='headings', height=15)
for col in columns:
    patient_listbox.heading(col, text=col)
    patient_listbox.column(col, width=120 if col != '電話' else 150, anchor='center')
patient_listbox.pack(pady=5)

# 掛號統計
count_label = tk.Label(root, text="目前候診總人數：0", font=('Arial', 12))
count_label.pack(pady=5)

# 操作按鈕
btn_frame = tk.Frame(root)
btn_frame.pack(pady=10)

tk.Button(btn_frame, text="新增掛號", command=open_add_window, width=15).grid(row=0, column=0, padx=10)
tk.Button(btn_frame, text="修改狀態", command=modify_status, width=15).grid(row=0, column=1, padx=10)
tk.Button(btn_frame, text="刪除掛號", command=delete_patient, width=15).grid(row=0, column=2, padx=10)

update_display()

root.mainloop()


# 功能1

In [3]:
import tkinter as tk
from tkinter import ttk, messagebox
from datetime import datetime
import csv
import sqlite3
import os

# 初始化 SQLite 資料庫
DB_PATH = 'patients.db'
def init_db():
    with sqlite3.connect(DB_PATH) as conn:
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS patients (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT,
                phone TEXT,
                status TEXT,
                time TEXT
            )
        ''')
        conn.commit()

# CSV 儲存
CSV_PATH = 'patients.csv'
def save_to_csv(patient):
    file_exists = os.path.isfile(CSV_PATH)
    with open(CSV_PATH, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(['姓名', '電話', '狀態', '掛號時間'])
        writer.writerow([patient['name'], patient['phone'], patient['status'], patient['time']])

# SQLite 儲存
def save_to_db(patient):
    with sqlite3.connect(DB_PATH) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO patients (name, phone, status, time) VALUES (?, ?, ?, ?)',
                       (patient['name'], patient['phone'], patient['status'], patient['time']))
        conn.commit()

# 遮罩姓名函數
def mask_name(name: str) -> str:
    if len(name) == 2:
        return name[0] + 'O'
    elif len(name) > 2:
        return name[0] + 'O' * (len(name) - 2) + name[-1]
    return name

# 更新候診顯示
def update_display():
    patient_listbox.delete(*patient_listbox.get_children())
    for idx, p in enumerate(patients):
        patient_listbox.insert('', 'end', values=(
            idx + 1, mask_name(p['name']), p['phone'], p['status'], p['time']))

# 新增掛號
def add_patient():
    win = tk.Toplevel(root)
    win.title("新增掛號")
    win.geometry("300x200")

    tk.Label(win, text="姓名").pack(pady=5)
    name_entry = tk.Entry(win)
    name_entry.pack()

    tk.Label(win, text="電話").pack(pady=5)
    phone_entry = tk.Entry(win)
    phone_entry.pack()

    def confirm():
        name = name_entry.get()
        phone = phone_entry.get()
        if name and phone:
            time_now = datetime.now().strftime('%H:%M:%S')
            patient = {'name': name, 'phone': phone, 'status': '候診中', 'time': time_now}
            patients.append(patient)
            save_to_csv(patient)
            save_to_db(patient)
            update_display()
            win.destroy()
        else:
            messagebox.showwarning("錯誤", "請填寫完整資訊")

    tk.Button(win, text="確認新增", command=confirm).pack(pady=10)

# 主程式
root = tk.Tk()
root.title("診所掛號系統 - 歷史紀錄儲存版")
root.geometry("800x500")

patients = []
init_db()

# 表格顯示
columns = ('序號', '姓名', '電話', '狀態', '掛號時間')
patient_listbox = ttk.Treeview(root, columns=columns, show='headings', height=15)
for col in columns:
    patient_listbox.heading(col, text=col)
    patient_listbox.column(col, anchor='center')
patient_listbox.pack(pady=10)

# 按鈕功能列
btn_frame = tk.Frame(root)
btn_frame.pack(pady=10)

tk.Button(btn_frame, text="新增掛號", command=add_patient, width=15).grid(row=0, column=0, padx=10)

update_display()
root.mainloop()

In [6]:
import tkinter as tk
from tkinter import ttk, messagebox
from datetime import datetime
import csv
import sqlite3
import os
import pyttsx3

# 初始化 SQLite 資料庫
DB_PATH = 'patients.db'
def init_db():
    with sqlite3.connect(DB_PATH) as conn:
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS patients (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT,
                phone TEXT,
                doctor TEXT,
                room TEXT,
                status TEXT,
                time TEXT
            )
        ''')
        conn.commit()

# CSV 儲存
CSV_PATH = 'patients.csv'
def save_to_csv(patient):
    file_exists = os.path.isfile(CSV_PATH)
    with open(CSV_PATH, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(['姓名', '電話', '醫師', '診間', '狀態', '掛號時間'])
        writer.writerow([patient['name'], patient['phone'], patient['doctor'], patient['room'], patient['status'], patient['time']])

# SQLite 儲存
def save_to_db(patient):
    with sqlite3.connect(DB_PATH) as conn:
        cursor = conn.cursor()
        cursor.execute('''INSERT INTO patients (name, phone, doctor, room, status, time) VALUES (?, ?, ?, ?, ?, ?)''',
                       (patient['name'], patient['phone'], patient['doctor'], patient['room'], patient['status'], patient['time']))
        conn.commit()

# 語音叫號
def announce(patient_id, name, room):
    engine = pyttsx3.init()
    engine.say(f"請 {patient_id} 號 {name} 到 {room} 報到")
    engine.runAndWait()

# 遮罩姓名函數
def mask_name(name: str) -> str:
    if len(name) == 2:
        return name[0] + 'O'
    elif len(name) > 2:
        return name[0] + 'O' * (len(name) - 2) + name[-1]
    return name

# 更新候診顯示（排序）
def update_display():
    patient_listbox.delete(*patient_listbox.get_children())
    sorted_patients = sorted(patients, key=lambda x: (status_order.get(x['status'], 99), x['time']))
    for idx, p in enumerate(sorted_patients):
        patient_listbox.insert('', 'end', values=(
            idx + 1, mask_name(p['name']), p['phone'], p['doctor'], p['room'], p['status'], p['time']))

# 新增掛號
def add_patient():
    win = tk.Toplevel(root)
    win.title("新增掛號")
    win.geometry("350x350")

    tk.Label(win, text="姓名").pack(pady=2)
    name_entry = tk.Entry(win)
    name_entry.pack()

    tk.Label(win, text="電話").pack(pady=2)
    phone_entry = tk.Entry(win)
    phone_entry.pack()

    tk.Label(win, text="醫師").pack(pady=2)
    doctor_combo = ttk.Combobox(win, values=["張醫師", "李醫師", "王醫師"])
    doctor_combo.pack()

    tk.Label(win, text="診間").pack(pady=2)
    room_combo = ttk.Combobox(win, values=["內科一診", "內科二診", "骨科"])
    room_combo.pack()

    def confirm():
        name = name_entry.get()
        phone = phone_entry.get()
        doctor = doctor_combo.get()
        room = room_combo.get()
        if name and phone and doctor and room:
            time_now = datetime.now().strftime('%H:%M:%S')
            patient = {'name': name, 'phone': phone, 'doctor': doctor, 'room': room, 'status': '候診中', 'time': time_now}
            patients.append(patient)
            save_to_csv(patient)
            save_to_db(patient)
            print_ticket(patient, len(patients))
            announce(len(patients), mask_name(name), room)
            update_display()
            win.destroy()
        else:
            messagebox.showwarning("錯誤", "請填寫所有欄位")

    tk.Button(win, text="確認新增", command=confirm).pack(pady=10)

# 列印報到單
def print_ticket(patient, number):
    ticket = f"""
    === 掛號報到單 ===
    編號：{number:03d}
    姓名：{mask_name(patient['name'])}
    電話：{patient['phone']}
    醫師：{patient['doctor']}
    診間：{patient['room']}
    時間：{patient['time']}
    =================
    """
    with open(f'ticket_{number:03d}.txt', 'w', encoding='utf-8') as f:
        f.write(ticket)
    messagebox.showinfo("報到單", ticket)

# 主程式
root = tk.Tk()
root.title("診所掛號系統（完整版）")
root.geometry("950x600")

patients = []
status_order = {"候診中": 0, "已到診": 1, "未到診": 2, "取消": 3}
init_db()

# 表格顯示
columns = ('序號', '姓名', '電話', '醫師', '診間', '狀態', '掛號時間')
patient_listbox = ttk.Treeview(root, columns=columns, show='headings', height=18)
for col in columns:
    patient_listbox.heading(col, text=col)
    patient_listbox.column(col, anchor='center', width=120)
patient_listbox.pack(pady=10)

# 按鈕列
btn_frame = tk.Frame(root)
btn_frame.pack(pady=10)

tk.Button(btn_frame, text="新增掛號", command=add_patient, width=15).grid(row=0, column=0, padx=10)

update_display()
root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\b.Application\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\user\AppData\Local\Temp\ipykernel_35504\2904151560.py", line 99, in confirm
    save_to_db(patient)
  File "C:\Users\user\AppData\Local\Temp\ipykernel_35504\2904151560.py", line 41, in save_to_db
    cursor.execute('''INSERT INTO patients (name, phone, doctor, room, status, time) VALUES (?, ?, ?, ?, ?, ?)''',
sqlite3.OperationalError: table patients has no column named doctor
Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\b.Application\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\user\AppData\Local\Temp\ipykernel_35504\2904151560.py", line 99, in confirm
    save_to_db(patient)
  File "C:\Users\user\AppData\Local\Temp\ipykernel_35504\2904151560.py", line

In [5]:
!pip install pyttsx3

   ---------------------------------------- 0.0/241.5 kB ? eta -:--:--
   ------------------------- -------------- 153.6/241.5 kB 3.1 MB/s eta 0:00:01
   ---------------------------------------  235.5/241.5 kB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 241.5/241.5 kB 2.5 MB/s eta 0:00:00


In [1]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import random
from itertools import permutations

# 每日排班邏輯
def generate_schedule(names):
    days = ['週一', '週二', '週三', '週四', '週五', '週六', '週日']
    schedule = {day: {'早班': [], '晚班': []} for day in days}
    rest_days = {name: set(random.sample(days, 2)) for name in names}
    available = {day: [] for day in days}

    for day in days:
        for name in names:
            if day not in rest_days[name]:
                available[day].append(name)

    for day in days:
        candidates = available[day][:]
        random.shuffle(candidates)
        if len(candidates) < 6:
            raise Exception(f"{day} 可排人數不足")
        day_shift = candidates[:3]
        night_shift = [p for p in candidates[3:] if p not in day_shift][:3]
        if len(night_shift) < 3:
            raise Exception(f"{day} 晚班人數不足")
        schedule[day]['早班'] = day_shift
        schedule[day]['晚班'] = night_shift

    return schedule, rest_days

# 顯示排班結果
def show_schedule():
    try:
        count = int(entry.get())
        if count < 6:
            messagebox.showerror("錯誤", "人數需至少 6 人")
            return
        names = [f"員工{i+1}" for i in range(count)]
        schedule, rest_days = generate_schedule(names)

        text.delete('1.0', tk.END)
        for day, shifts in schedule.items():
            text.insert(tk.END, f"{day}\n")
            text.insert(tk.END, f"  早班: {', '.join(shifts['早班'])}\n")
            text.insert(tk.END, f"  晚班: {', '.join(shifts['晚班'])}\n\n")

        text.insert(tk.END, f"\n休假清單：\n")
        for name, rests in rest_days.items():
            text.insert(tk.END, f"{name} 休假: {', '.join(rests)}\n")

    except ValueError:
        messagebox.showerror("輸入錯誤", "請輸入有效的人數")
    except Exception as e:
        messagebox.showerror("排班錯誤", str(e))

# GUI
root = tk.Tk()
root.title("排班系統")
root.geometry("600x600")

frame = tk.Frame(root)
frame.pack(pady=10)

label = tk.Label(frame, text="輸入人數（至少6人）:")
label.pack(side=tk.LEFT)

entry = tk.Entry(frame)
entry.pack(side=tk.LEFT, padx=5)

btn = tk.Button(frame, text="產生排班表", command=show_schedule)
btn.pack(side=tk.LEFT)

text = tk.Text(root, font=("Courier", 12), width=70, height=25)
text.pack(pady=10)

root.mainloop()


In [5]:
status = int(input())
match status:
    case 400:
        print ("狀態是400")
    case 404:
        print ("狀態是404")
    case 418:
        print ("狀態是418")
    case _:
        print ("狀態歸類到其他")


狀態是400
